In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [ ]:
OPEN_AI_API_KEY = "API_KEY"

In [3]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0, openai_api_key = OPEN_AI_API_KEY)

In [4]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, chunk_overlap=50)

splits = text_splitter.split_documents(blog_docs)

vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding= OpenAIEmbeddings(openai_api_key = OPEN_AI_API_KEY))

retriever = vectorstore.as_retriever()

In [5]:
# HyDE document genetation
template = """ Please write a scientific paper passage to answer the question
Question: {question}
Passage: """

prompt_hyde = ChatPromptTemplate.from_template(template)

generate_docs_for_retrieval = (
    prompt_hyde | 
    ChatOpenAI(temperature=0, openai_api_key = OPEN_AI_API_KEY) | 
    StrOutputParser()
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

'Task decomposition for LLM agents refers to the process of breaking down a complex task into smaller, more manageable sub-tasks that can be executed sequentially or in parallel. This approach allows LLM agents to efficiently tackle large and complex problems by dividing them into smaller, more manageable components. By decomposing tasks, LLM agents can allocate resources more effectively, reduce computational complexity, and improve overall performance. Additionally, task decomposition enables LLM agents to leverage parallel processing capabilities, leading to faster and more efficient problem-solving. Overall, task decomposition is a crucial strategy for enhancing the capabilities and performance of LLM agents in handling complex tasks.'

In [6]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever
retrieved_docs = retrieval_chain.invoke({"question":question})

retrieved_docs

[Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via 

In [7]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retrieved_docs, "question":question})

'Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals. This enables the agent to efficiently handle complex tasks by dividing them into smaller and simpler steps. Task decomposition can be achieved through techniques such as Chain of Thought (CoT) and Tree of Thoughts, as well as through simple prompting or task-specific instructions.'